Libraries needed!

In [20]:
import pandas as pd
import numpy as np
from Bio import Entrez, SeqIO
from Bio.SeqFeature import FeatureLocation
import time
import os

Match Identified BGCs to locus tags

In [17]:
def extract_prefix(value):
    """Extract the prefix (numbers before first underscore) from the value."""
    try:
        parts = value.split('_')
        if len(parts) >= 2:
            return f"{parts[0]}_{parts[1]}"
        return np.nan
    except:
        return np.nan

def match_bgc_codes(file_path):
    # Load the Excel file
    data = pd.read_excel(file_path)
    
    # Convert columns to string and replace nan with empty string
    data['BGC_code'] = data['BGC_code'].fillna('').astype(str)
    data['HuMi_BGC'] = data['HuMi_BGC'].fillna('').astype(str)
    
    # Create a dictionary to store BGC_code and its corresponding locus tags
    bgc_info = {}
    for _, row in data.iterrows():
        bgc_info[row['BGC_code']] = {
            'Bginning_locus_tag': row['Bginning locus tag'],
            'Ending_locus_tag': row['Ending locus tag']
        }
    
    # Get unique values from both columns, removing empty strings
    humi_values = data[data['HuMi_BGC'].str.strip() != '']['HuMi_BGC'].unique()
    bgc_values = data[data['BGC_code'].str.strip() != '']['BGC_code'].unique()
    
    # Create a list to store all matches
    all_matches = []
    
    # Process each HuMi_BGC value
    total_humi = len(humi_values)
    print(f"Starting to process {total_humi} unique HuMi_BGC entries against {len(bgc_values)} unique BGC_codes")
    
    for idx, humi_value in enumerate(humi_values, 1):
        if idx % 100 == 0:  # Progress indicator
            print(f"Processing HuMi_BGC {idx}/{total_humi}")
            
        humi_prefix = extract_prefix(humi_value)
        if pd.isna(humi_prefix):
            continue
        
        # Check against ALL unique BGC_codes
        for bgc_code in bgc_values:
            bgc_prefix = extract_prefix(bgc_code)
            if pd.isna(bgc_prefix):
                continue
                
            if humi_prefix == bgc_prefix:
                match_info = {
                    'HuMi_BGC': humi_value,
                    'BGC_code': bgc_code,
                    'Matched_Prefix': humi_prefix,
                    'Bginning_locus_tag': bgc_info[bgc_code]['Bginning_locus_tag'],
                    'Ending_locus_tag': bgc_info[bgc_code]['Ending_locus_tag']
                }
                all_matches.append(match_info)
    
    # Convert matches to DataFrame
    matched_data = pd.DataFrame(all_matches)
    
    # Remove any exact duplicates while preserving all valid matches
    matched_data = matched_data.drop_duplicates()
    
    # Reorder columns for better readability
    column_order = ['HuMi_BGC', 'BGC_code', 'Bginning_locus_tag', 'Ending_locus_tag', 'Matched_Prefix']
    matched_data = matched_data[column_order]
    
    # Print statistics
    print(f"\nMatching Statistics:")
    print(f"Total unique HuMi_BGC entries processed: {len(humi_values)}")
    print(f"Total unique BGC_code entries checked: {len(bgc_values)}")
    print(f"Total matches found: {len(matched_data)}")
    print(f"Number of unique HuMi_BGC in matches: {len(matched_data['HuMi_BGC'].unique())}")
    
    return matched_data

# Use the function
file_path = "Data/Sup_data.xlsx"
matched_data = match_bgc_codes(file_path)

# Save the results
output_file = "Data/Matched_BGCs_tags.xlsx"
matched_data.to_excel(output_file, index=False)
print(f"\nFile '{output_file}' created successfully!")

# Display a few example matches
print("\nFirst few matches (showing all columns):")
print(matched_data.head())

Starting to process 3118 unique HuMi_BGC entries against 13898 unique BGC_codes
Processing HuMi_BGC 100/3118
Processing HuMi_BGC 200/3118
Processing HuMi_BGC 300/3118
Processing HuMi_BGC 400/3118
Processing HuMi_BGC 500/3118
Processing HuMi_BGC 600/3118
Processing HuMi_BGC 700/3118
Processing HuMi_BGC 800/3118
Processing HuMi_BGC 900/3118
Processing HuMi_BGC 1000/3118
Processing HuMi_BGC 1100/3118
Processing HuMi_BGC 1200/3118
Processing HuMi_BGC 1300/3118
Processing HuMi_BGC 1400/3118
Processing HuMi_BGC 1500/3118
Processing HuMi_BGC 1600/3118
Processing HuMi_BGC 1700/3118
Processing HuMi_BGC 1800/3118
Processing HuMi_BGC 1900/3118
Processing HuMi_BGC 2000/3118
Processing HuMi_BGC 2100/3118
Processing HuMi_BGC 2200/3118
Processing HuMi_BGC 2300/3118
Processing HuMi_BGC 2400/3118
Processing HuMi_BGC 2500/3118
Processing HuMi_BGC 2600/3118
Processing HuMi_BGC 2700/3118
Processing HuMi_BGC 2800/3118
Processing HuMi_BGC 2900/3118
Processing HuMi_BGC 3000/3118
Processing HuMi_BGC 3100/3118

**Downloading GBK files for first 30 BGCs**

In [21]:
from Bio import Entrez, SeqIO
from Bio.SeqFeature import FeatureLocation
from Bio.SeqRecord import SeqRecord
import pandas as pd
import time
import os

def setup_email():
    Entrez.email = "anju040301@gmail.com"

def search_locus_tag(locus_tag):
    """
    Search for a specific locus tag and return hits info.
    """
    try:
        search_handle = Entrez.esearch(db="nucleotide", 
                                     term=f'"{locus_tag}"[Locus Tag]',
                                     retmax=10)
        search_results = Entrez.read(search_handle)
        search_handle.close()
        
        return {
            'total_hits': int(search_results["Count"]),
            'ids': search_results["IdList"]
        }
    except Exception as e:
        print(f"Error searching for {locus_tag}: {str(e)}")
        return {'total_hits': 0, 'ids': []}

def has_contig_information(record):
    """
    Check if GenBank record already contains contig information.
    """
    if not record:
        return False
    
    # Check if sequence data exists
    if not record.seq or len(record.seq) == 0:
        return False
    
    # Check if sequence contains actual nucleotides (not just gaps or unknown bases)
    seq_str = str(record.seq)
    nucleotides = set('ATGCatgc')
    if not any(base in nucleotides for base in seq_str):
        return False
    
    return True

def fetch_sequence_data(seq_id):
    """
    Fetch GenBank record and FASTA data if needed.
    Returns: (gbk_record, fasta_record, needs_fasta)
    """
    try:
        # First fetch GenBank record
        gbk_handle = Entrez.efetch(db="nucleotide", id=seq_id, rettype="gbwithparts", retmode="text")
        gbk_record = SeqIO.read(gbk_handle, "genbank")
        gbk_handle.close()
        
        # Check if we need FASTA data
        needs_fasta = not has_contig_information(gbk_record)
        
        if needs_fasta:
            print("GenBank record lacks contig information. Fetching FASTA data...")
            fasta_handle = Entrez.efetch(db="nucleotide", id=seq_id, rettype="fasta", retmode="text")
            fasta_record = SeqIO.read(fasta_handle, "fasta")
            fasta_handle.close()
        else:
            print("GenBank record contains complete sequence information. Skipping FASTA fetch.")
            fasta_record = None
            
        return gbk_record, fasta_record, needs_fasta
    
    except Exception as e:
        print(f"Error fetching sequence data for {seq_id}: {str(e)}")
        return None, None, False

def merge_records(gbk_record, fasta_record, start_pos, end_pos, needs_fasta):
    """
    Create BGC record, merging with FASTA data if needed.
    """
    try:
        if needs_fasta and fasta_record:
            # Use FASTA sequence
            bgc_sequence = fasta_record.seq[start_pos:end_pos]
        else:
            # Use GenBank sequence
            bgc_sequence = gbk_record.seq[start_pos:end_pos]
        
        # Create new record
        merged_record = SeqRecord(
            seq=bgc_sequence,
            id=f"{gbk_record.id}_BGC",
            description=f"BGC extracted from positions {start_pos} to {end_pos}",
            annotations={"molecule_type": "DNA"}
        )
        
        # Transfer relevant features
        for feature in gbk_record.features:
            if feature.location.start >= start_pos and feature.location.end <= end_pos:
                new_location = FeatureLocation(
                    feature.location.start - start_pos,
                    feature.location.end - start_pos
                )
                feature.location = new_location
                merged_record.features.append(feature)
        
        return merged_record
    
    except Exception as e:
        print(f"Error merging records: {str(e)}")
        return None

def fetch_and_extract_bgc(start_locus, end_locus, bgc_code):
    """
    Fetch and extract BGC region.
    """
    print(f"\nSearching hits for BGC {bgc_code}:")
    start_results = search_locus_tag(start_locus)
    if not start_results or 'ids' not in start_results:
        print(f"No valid results for start locus {start_locus}")
        return None, 0, 0
        
    print(f"Start locus {start_locus}: {start_results['total_hits']} hits")
    time.sleep(1)
    
    end_results = search_locus_tag(end_locus)
    if not end_results or 'ids' not in end_results:
        print(f"No valid results for end locus {end_locus}")
        return None, start_results['total_hits'], 0
        
    print(f"End locus {end_locus}: {end_results['total_hits']} hits")
    
    if not start_results['ids'] or not end_results['ids']:
        print("No results found for one or both locus tags")
        return None, start_results['total_hits'], end_results['total_hits']

    for seq_id in start_results['ids']:
        try:
            print(f"Fetching records for ID: {seq_id}")
            gbk_record, fasta_record, needs_fasta = fetch_sequence_data(seq_id)
            
            if not gbk_record:
                continue
            
            # Find positions of locus tags
            start_pos = None
            end_pos = None
            
            for feature in gbk_record.features:
                if 'locus_tag' in feature.qualifiers:
                    locus = feature.qualifiers['locus_tag'][0]
                    if locus == start_locus:
                        start_pos = feature.location.start
                    elif locus == end_locus:
                        end_pos = feature.location.end
            
            if start_pos is None or end_pos is None:
                continue
            
            print(f"Found both locus tags in record {seq_id}")
            bgc_record = merge_records(gbk_record, fasta_record, start_pos, end_pos, needs_fasta)
            
            if bgc_record:
                return bgc_record, start_results['total_hits'], end_results['total_hits'], needs_fasta
            
        except Exception as e:
            print(f"Error processing record {seq_id}: {str(e)}")
            continue
    
    return None, start_results['total_hits'], end_results['total_hits'], False
    
def determine_bgc_category(bgc_record, needs_fasta):
    """
    Determine BGC category based on specified criteria.
    Returns: (main_category, sub_category)
    """
    if not bgc_record:
        return "BGC not found", "N/A"
        
    if len(bgc_record.seq) == 0:
        return "BGC found without contig", "No DNA sequence"
        
    if needs_fasta:
        return "BGC found with added FASTA", "DNA sequence added from FASTA"
        
    # BGC found with contig
    if len(bgc_record.features) <= 2:
        return "BGC found with contig", "Only BGC information"
    else:
        return "BGC found with contig", "Additional information"

def analyze_bgc_records(matched_data_file, output_dir="Data/GBK_loadedFiles/", num_rows=30):
    """Analyze BGC records and save results."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    df = pd.read_excel(matched_data_file)
    df = df.head(num_rows)
    
    results = []
    
    for idx, row in df.iterrows():
        bgc_code = row['BGC_code']
        start_locus = row['Bginning_locus_tag']
        end_locus = row['Ending_locus_tag']
        
        if pd.isna(start_locus) or pd.isna(end_locus):
            print(f"Skipping {bgc_code}: Missing locus tags")
            results.append({
                'BGC_code': bgc_code,
                'status': 'missing_tags',
                'start_locus_hits': 0,
                'end_locus_hits': 0,
                'sequence_length': 0,
                'feature_count': 0,
                'main_category': 'BGC not found',
                'sub_category': 'Missing locus tags',
                'total_hits': 0
            })
            continue
            
        print(f"\nProcessing {idx+1}/{num_rows}: {bgc_code}")
        
        result = fetch_and_extract_bgc(start_locus, end_locus, bgc_code)
        if result is None or len(result) < 4:
            results.append({
                'BGC_code': bgc_code,
                'status': 'not_found',
                'start_locus_hits': 0,
                'end_locus_hits': 0,
                'sequence_length': 0,
                'feature_count': 0,
                'main_category': 'BGC not found',
                'sub_category': 'Search failed',
                'total_hits': 0
            })
            continue
            
        bgc_record, start_hits, end_hits, needs_fasta = result
        
        if bgc_record:
            # Determine categories
            main_category, sub_category = determine_bgc_category(bgc_record, needs_fasta)
            
            # Save BGC file if it's a complete record
            if main_category in ["BGC found with contig", "BGC found with added FASTA"]:
                gbk_file = os.path.join(output_dir, f"{bgc_code}.gbk")
                SeqIO.write(bgc_record, gbk_file, "genbank")
            
            results.append({
                'BGC_code': bgc_code,
                'status': 'found',
                'start_locus_hits': start_hits,
                'end_locus_hits': end_hits,
                'sequence_length': len(bgc_record.seq),
                'feature_count': len(bgc_record.features),
                'main_category': main_category,
                'sub_category': sub_category,
                'total_hits': start_hits + end_hits
            })
        else:
            results.append({
                'BGC_code': bgc_code,
                'status': 'error',
                'start_locus_hits': start_hits,
                'end_locus_hits': end_hits,
                'sequence_length': 0,
                'feature_count': 0,
                'main_category': 'BGC not found',
                'sub_category': 'Processing error',
                'total_hits': start_hits + end_hits
            })
        
        time.sleep(1)
    
    return results

def save_results(results, output_file="Data/bgc_analysis.csv"):
    """Save analysis results to CSV file and print detailed summary."""
    df = pd.DataFrame(results)
    df.to_csv(output_file, index=False)
    
    print("\n=== Results Summary ===")
    
    # Summary by main category
    print("\nMain Category Summary:")
    main_category_counts = df['main_category'].value_counts()
    for category, count in main_category_counts.items():
        print(f"{category}: {count} BGCs")
        # Get subcategories for this main category
        sub_cats = df[df['main_category'] == category]['sub_category'].value_counts()
        for sub_cat, sub_count in sub_cats.items():
            print(f"  - {sub_cat}: {sub_count} BGCs")
    
    # Detailed hits information
    print("\nDetailed BGC Information:")
    for result in results:
        print(f"\nBGC: {result['BGC_code']}")
        print(f"  Main Category: {result['main_category']}")
        print(f"  Sub Category: {result['sub_category']}")
        print(f"  Start Locus Hits: {result['start_locus_hits']}")
        print(f"  End Locus Hits: {result['end_locus_hits']}")
        print(f"  Total Hits: {result['total_hits']}")
        if result['status'] == 'found':
            print(f"  Sequence Length: {result['sequence_length']}")
            print(f"  Feature Count: {result['feature_count']}")

def main():
    setup_email()
    matched_data_file = "Data/Matched_BGCs_tags.xlsx"
    results = analyze_bgc_records(matched_data_file)
    save_results(results)

if __name__ == "__main__":
    main()


Processing 1/30: 643901933_0

Searching hits for BGC 643901933_0:
Start locus HMPREF0530_2701: 4 hits
End locus HMPREF0530_2706: 4 hits
Fetching records for ID: 238694749
GenBank record contains complete sequence information. Skipping FASTA fetch.
Fetching records for ID: 238618246
GenBank record contains complete sequence information. Skipping FASTA fetch.
Found both locus tags in record 238618246

Processing 2/30: 643010393_0

Searching hits for BGC 643010393_0:
Start locus CAPSP0001_2757: 2 hits
End locus CAPSP0001_2773: 3 hits
Fetching records for ID: 213955701
Error fetching sequence data for 213955701: Sequence content is undefined
Fetching records for ID: 213953969
GenBank record contains complete sequence information. Skipping FASTA fetch.
Found both locus tags in record 213953969

Processing 3/30: 643892130_0

Searching hits for BGC 643892130_0:
Start locus HMPREF0294_1046: 4 hits
End locus HMPREF0294_1051: 3 hits
Fetching records for ID: 237753624
GenBank record contains com